<a href="https://colab.research.google.com/github/MakkuranoNeko/Portfolio/blob/main/%D0%91%D0%B0%D0%BD%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B5_%D0%BA%D0%B0%D1%80%D1%82%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Дан сэмпл данных, в котором находится информация о логинах пользователей, загрузке денежных средств на банковскую карту и трейдинге(если он необходим). 
Трейдинг происходит в случае если валюта денежных средств, которые пользователь хочет загрузить на карту, не совпадает с валютой, в которой была выпущена карта. 
Необходимо проанализировать сэмпл данных и ответить на следующие вопросы:
* Среднее кол-во средств загружаемых на банковскую карту? (по валютам и по типам карт)
* Среднее кол-во загрузок на карты в день?
* Сколько всего посетителей за все время(visitors)?
* Сколько пользователей загружали средства на карту(customers)?
* Какой CR(конверсия) из visitors в customers?
* Посчитайте метрики ARPU(среднее revenue на пользователя), ARPPU(среднее revenue на платящего пользователя), ARPP (среднее revenue на покупку), APPPU (среднее кол-во покупок на платящего пользователя). При расчетах метрик необходимо учитывать как загрузки на карту так и трейдинг
* Сколько суммарно заработала компания (revenue)
* Есть ли различия между разными девайсами? (PC/MOB) Опишите подробнее в чем различия, если они есть.
* Какой % транзакций «Loaded Debit Card» происходит без трейдинга?

Дополнительно: Попытаться найти интересные факты из предложенных данных, которые могут быть полезны для продукта или бизнеса. (задание необязательное)

# **Предобработка данных**

In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px

In [ ]:
data = pd.read_csv('test_data_set.csv', sep = ',')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381707 entries, 0 to 381706
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            381707 non-null  object 
 1   ts              381707 non-null  int64  
 2   user_id         381707 non-null  int64  
 3   category_event  380652 non-null  object 
 4   event           381707 non-null  object 
 5   device_type     378190 non-null  object 
 6   reference       14632 non-null   object 
 7   revenue         14632 non-null   float64
 8   amount          8760 non-null    float64
 9   currency        8760 non-null    object 
 10  card_id         8760 non-null    object 
 11  card_type       8760 non-null    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 34.9+ MB


In [ ]:
data.describe()

,ts,user_id,revenue,amount
count,3.817070e+05,381707.000000,14632.000000,8760.000000
mean,1.512308e+09,303323.211927,77.443505,3979.862863
std,7.308125e+05,106359.194624,194.330277,9587.345437
min,1.510790e+09,5.000000,-2975.300000,8.440000
25%,1.511776e+09,237581.000000,4.363000,214.275000
50%,1.512413e+09,349718.000000,17.207000,840.280000
75%,1.512959e+09,385724.000000,60.564500,3193.900000
max,1.513382e+09,421755.000000,2510.200000,83543.000000


In [ ]:
data.head(1)

,date,ts,user_id,category_event,event,device_type,reference,revenue,amount,currency,card_id,card_type
0,2017-11-16 00:00:06,1510790406,323681,Login,Logged In,PC,NaN,NaN,NaN,NaN,NaN,NaN


Как видно из общей информации о дата сете, стоблец даты нужно привести в стандартный формат datetime. Также добавим столбец для указания дня и месяца.

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
data['time'] = data['date'].dt.time

In [ ]:
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month

Так как здесь много номинативных переменных, то посмотрим какие значения они принимают.

In [ ]:
list_columns = list(data.columns)
print(list_columns)

['date', 'ts', 'user_id', 'category_event', 'event', 'device_type', 'reference', 'revenue', 'amount', 'currency', 'card_id', 'card_type', 'time', 'day', 'month']


Для этого выделим список всех названий колонок для номинативных переменных:

In [ ]:
short_list_columns = ['category_event', 'event', 'device_type', 
                      'currency', 'card_type']

In [ ]:
unique_values = {kategory: list(data[kategory].unique())
                for kategory in short_list_columns} 
print(unique_values)

{'category_event': ['Login', 'Debit Card Operations', 'Trading', nan], 'event': ['Logged In', 'Loaded Debit Card', 'Traded Card Load'], 'device_type': ['PC', 'MOB', nan], 'currency': [nan, 'EUR', 'USD', 'GBP'], 'card_type': [nan, 'plastic', 'virtual']}


Стоит посмотреть как связан столбец event и revenue. Есть ли пропущенные значения в переводах.

In [ ]:
data[data['event'].isin(['Loaded Debit Card', 'Traded Card Load'])].revenue.unique()

array([11.32 , 27.758,  4.232, ...,  5.595, 18.846, 90.64 ])

In [ ]:
data[data['revenue'].isna()].category_event.unique()

array(['Login', nan], dtype=object)

Пустые значения есть только при логине. Значит данные о пополнении карт полные.

# **Ответы на основные вопросы**

**1.Среднее кол-во средств загружаемых на банковскую карту? (по валютам и по типам карт)**

In [ ]:
pd.pivot_table(data, values='amount', index='currency', columns='card_type', aggfunc='mean')

card_type,plastic,virtual
currency,,
EUR,5085.874674,1753.118849
GBP,4814.375244,1061.469258
USD,7029.681152,1264.989707


Из таблицы можно сделать вывод, что основная часть средст поступает в пластиковые карты, и лишь часть на виртуальные карты. Также стоит отметить, что основная часть денег переводится в USD.

**2. Среднее кол-во загрузок на карты в день?** 

In [ ]:
temp_data = data[data['event'].isin(['Loaded Debit Card', 'Traded Card Load'])]

In [ ]:
pd.pivot_table(temp_data, values='revenue',index=['month', 'day'], aggfunc='count').mean()

revenue    487.733333
dtype: float64

Среднее число транзакций в день:  487.733333

Также поставленную задачу можно было бы решить прямо посчитав количество дней, округлив их число в большую сторону.

In [ ]:
temp_data.date.max() - temp_data.date.min()

Timedelta('29 days 23:25:41')

In [ ]:
count_days = 29
print(count_days)

29


In [ ]:
len(temp_data) / (count_days + 1)

487.73333333333335

**3. Сколько всего посетителей за все время(visitors)?**

In [ ]:
visitors = len(data.user_id.unique())

In [ ]:
print('Всего посетителей было:', visitors)

Всего посетителей было: 79194


**4. Сколько пользователей загружали средства на карту(customers)?**

In [ ]:
customers = len(temp_data.user_id.unique())

In [ ]:
print('Число customers:', customers)

Число customers: 3048


 **5. Какой CR(конверсия) из visitors в customers?**

In [ ]:
CR = (customers / visitors) * 100

In [ ]:
print("CR(конверсия) из visitors в customers: ", round(CR,2), '%.')

CR(конверсия) из visitors в customers:  3.85 %.


**6. Посчитайте метрики ARPU(среднее revenue на пользователя), ARPPU(среднее revenue на платящего пользователя), ARPP (среднее revenue на покупку), APPPU (среднее кол-во покупок на платящего пользователя). При расчетах метрик необходимо учитывать как загрузки на карту так и трейдинг**

In [ ]:
ARPU = (data.revenue.sum() / len(data))
print("ARPU(среднее revenue на пользователя):",round(ARPU, 2))

ARPU(среднее revenue на пользователя): 2.97


In [ ]:
ARPPU = (data.revenue.sum() / len(temp_data.user_id.unique()))
print("ARPPU(среднее revenue на платящего пользователя):",round(ARPPU, 2))

ARPPU(среднее revenue на платящего пользователя): 371.77


In [ ]:
ARPP = (data.revenue.sum() / len(temp_data))
print('ARPP (среднее revenue на покупку):',round(ARPP, 2))

ARPP (среднее revenue на покупку): 77.44


In [ ]:
APPPU = len(temp_data) / len(temp_data.user_id.unique())
print('APPPU (среднее кол-во покупок на платящего пользователя):',round(APPPU, 2))

APPPU (среднее кол-во покупок на платящего пользователя): 4.8


**7. Сколько суммарно заработала компания (revenue)**

In [ ]:
print('Компания суммарно заработала:',data.revenue.sum())

Компания суммарно заработала: 1133153.3714


**8. Есть ли различия между разными девайсами? (PC/MOB) Опишите подробнее в чем различия, если они есть.**
    

In [ ]:
data_pc = data[data['device_type'] == 'PC']
data_mob = data[data['device_type'] == 'MOB'] 

In [ ]:
print(round(len(data_mob) / len(data) * 100,2),'% приходится на мобильные устройства и',
      round(len(data_pc) / len(data) * 100,2), '% на компьютеры.')
print('Часть данных,', round((len(data) - len(data_mob) - len(data_pc)) / len(data) 
      * 100,2), '% с пропущенными значениями.')

22.73 % приходится на мобильные устройства и 76.35 % на компьютеры.
Часть данных, 0.92 % с пропущенными значениями.


Посмотрим на процентное соотношение видов операций на каждом из устройств. 

In [ ]:
pd.pivot_table(data, values='revenue', index='device_type', 
               columns='category_event', aggfunc=lambda x: len(x) / len(data) * 100)

category_event,Debit Card Operations,Login,Trading
device_type,,,
MOB,0.574524,21.481398,0.400831
PC,1.680084,73.547512,1.117873


Различия в типах работы с картой нет.

Пользуются ли чаще виртуальной или пластиковой картой на каждом из типов устройств?

In [ ]:
table1 = pd.pivot_table(data, values='revenue', index='device_type', 
               columns='card_type', aggfunc='count')
table1

card_type,plastic,virtual
device_type,,
MOB,1785,408
PC,4096,2317


Можно отметить, что с мобильных устройств чаще заходят для того чтобы работать с пластиковой картой, в то время как треть операций с компьютера происходит именно с виртуальной картой.

In [ ]:

pd.pivot_table(data, values='revenue', index='device_type', 
               columns='currency', 
               aggfunc='count')

currency,EUR,GBP,USD
device_type,,,
MOB,1273,661,259
PC,3580,1493,1340


Рассматривая тип валют, с которым работают клиенты, можно заметить, что меньше всего транзакций с USD было сделано с мобильных устройств.

In [ ]:
pd.pivot_table(temp_data, values='revenue', index='device_type', columns=['event','currency'], aggfunc='mean')

event       Loaded Debit Card                      
currency                  EUR        GBP        USD
device_type                                        
MOB                 71.229131  88.506842  38.790579
PC                  92.242152  96.259570  55.214007

В то время как средние суммы транзакций в среднем остаются неизменными для каждой из валют, что для мобильных устройств, что для компьютеров.

In [ ]:
pd.pivot_table(temp_data, values='revenue', index='device_type', columns=['event'], aggfunc='mean')

event,Loaded Debit Card,Traded Card Load
device_type,,
MOB,72.605776,64.095747
PC,85.440389,73.902899


**9. Какой % транзакций «Loaded Debit Card» происходит без трейдинга?**


In [ ]:
print(round(len(data[(data['event'] == 'Loaded Debit Card')]) / (len(data[(data['event'] == 'Loaded Debit Card')]) 
+ len(data[(data['event'] == 'Traded Card Load')])) * 100, 2),'% транзакций "Loaded Debit Card" происходит без трейдинга')

59.87 % транзакций "Loaded Debit Card" происходит без трейдинга



# **Дополнительно: Попытаться найти интересные факты из предложенных данных, которые могут быть полезны для продукта или бизнеса. (задание необязательное)**

Посмотрим еще раз на статистику по всем численным переменным на предмет выбросов.

In [ ]:
data.describe()

,ts,user_id,revenue,amount,day,month
count,3.817070e+05,381707.000000,14632.000000,8760.000000,381707.000000,381707.000000
mean,1.512308e+09,303323.211927,77.443505,3979.862863,14.542191,11.614780
std,7.308125e+05,106359.194624,194.330277,9587.345437,8.518740,0.486648
min,1.510790e+09,5.000000,-2975.300000,8.440000,1.000000,11.000000
25%,1.511776e+09,237581.000000,4.363000,214.275000,7.000000,11.000000
50%,1.512413e+09,349718.000000,17.207000,840.280000,13.000000,12.000000
75%,1.512959e+09,385724.000000,60.564500,3193.900000,22.000000,12.000000
max,1.513382e+09,421755.000000,2510.200000,83543.000000,30.000000,12.000000


In [ ]:
fig = px.box(data, y='revenue')
fig.show()

Интересный факт, есть выбросы в данных о заработке компании. Существует несколько значений revenue с отрицательным значением.

In [ ]:
data[data['revenue']<0]

,date,ts,user_id,category_event,event,device_type,reference,revenue,amount,currency,card_id,card_type,time,day,month
11168,2017-11-17 10:39:06,1510915146,350709,Trading,Traded Card Load,MOB,CL202946171117,-0.8000,NaN,NaN,NaN,NaN,10:39:06,17,11
16993,2017-11-17 21:24:04,1510953844,268650,Trading,Traded Card Load,PC,CL203279171117,-3.2520,NaN,NaN,NaN,NaN,21:24:04,17,11
17736,2017-11-17 23:54:03,1510962843,233,Trading,Traded Card Load,PC,CL203131171117,-45.8060,NaN,NaN,NaN,NaN,23:54:03,17,11
18088,2017-11-18 01:57:01,1510970221,332627,Trading,Traded Card Load,PC,CL202944171117,-29.9460,NaN,NaN,NaN,NaN,01:57:01,18,11
41837,2017-11-21 04:54:04,1511240044,247105,Trading,Traded Card Load,PC,CL204940211117,-650.7600,NaN,NaN,NaN,NaN,04:54:04,21,11
49948,2017-11-21 22:36:04,1511303764,270918,Trading,Traded Card Load,MOB,CL205380211117,-28.0460,NaN,NaN,NaN,NaN,22:36:04,21,11
50230,2017-11-21 23:27:04,1511306824,359727,Trading,Traded Card Load,PC,CL205388211117,-0.6870,NaN,NaN,NaN,NaN,23:27:04,21,11
50271,2017-11-21 23:36:04,1511307364,198479,Trading,Traded Card Load,PC,CL205387211117,-5.4900,NaN,NaN,NaN,NaN,23:36:04,21,11
50483,2017-11-22 00:36:03,1511310963,244381,Trading,Traded Card Load,MOB,CL205347211117,-1.5700,NaN,NaN,NaN,NaN,00:36:03,22,11
50499,2017-11-22 00:42:03,1511311323,53024,Trading,Traded Card Load,PC,CL205354211117,-56.9920,NaN,NaN,NaN,NaN,00:42:03,22,11


Все эти операции были сделаны как трейдинг.

**Проверим когда чаще всего логинятся в приложении на PC и на мобильных телефонах.**

Для этого добавим колонку с часами и построим сводную таблицу данных о логине в приложение

In [ ]:
data['hour'] = data['date'].dt.hour

In [ ]:
pivot = data.pivot_table(index=['hour', 'device_type'], values='date', aggfunc='count').reset_index()

Теперь построим бар плот из полученных значений:

In [ ]:
px.bar(pivot, x='hour', y='date', color='device_type')

**Вывод:**
1. Для пользователей логинящихся через PC выделяется явный спад активности ночью с 0 до 6 утра. Что говорит о том, что количество персонала поддержки в ночные смены должно быть меньше чем в дневные.
2. С 10 до 20 часов выделяется самая активная фаза и значит количество персонала поддержки должно быть максимально.
3. Для мобильного приложения тоже виден спад, но в целом график пологий и здесь уже нет такого резкого перепада количества логинов.
4. В данных о заработке компании есть отрицательные выбросы, несколько значений revenue с отрицательным значением. Все эти операции проводились как трейдинг. Возможно система неправильно обрабатывает такие запросы.